# Выбор оптимальной конфигурации модели

In [1]:
# standard libraries
import pandas as pd
import sys
import os
import yaml
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.float_format','{:_.2f}'.format)
if 'drive' not in os.listdir():
    from google.colab import drive
    drive.mount('/content/drive')

path_to_folder = '/content/drive/MyDrive/psb_hack'

# Выбор лучшей модели по метрикам по продукту

In [2]:
main_metrics = ['precision', 'recall', 'f1', 'roc-auc original', 'roc-auc_hack']
test_set_name = 'val_oot'

def read_metric_table(file_name):
    table = pd.read_excel(file_name)
    table['config_model'] = file_name.split('/')[-1].split('.')[0].replace('_metrics','')
    return table

# Загрзука метрик

In [3]:
# файлы с метриками по модели суммы кредита
metrics = os.listdir(os.path.join(path_to_folder, 'metrics'))
# соберем их в единый файл с указанием модели
all_metrics = pd.concat(list(map(lambda x: read_metric_table(os.path.join(path_to_folder,'metrics', x)),metrics))).reset_index(drop=True)

In [4]:
all_metrics.sample(5)

,dataset,target,group,measurement,N,accuracy,precision,recall,f1,roc-auc original,roc-auc_hack,pr-auc,target_label_count_0,target_label_count_1,pred_label_count_0,pred_label_count_1,auc,config_model
13,oot_val,target,Источник,ostrovok.ru,136,0.90,0.33,0.08,0.12,0.73,0.53,0.21,123,13,133,3.00,0.73,psb_hack_lama_v541
7,train,target,Источник,Яндекс.Путешествия,629,0.95,1.00,0.87,0.93,0.99,0.93,0.99,381,248,414,215.00,0.99,psb_hack_lama_v541
26,oof_val,target,Источник,Яндекс.Путешествия,251,0.94,1.00,0.81,0.90,0.95,0.91,0.95,165,86,181,70.00,0.95,psb_hack_lama_v541
6,train,target,Источник,Программа лояльности,304,0.93,1.00,0.21,0.35,0.89,0.61,0.68,276,28,298,6.00,0.89,psb_hack_lama_v541
15,oot_val,target,Источник,Официальный сайт,2177,0.92,0.99,0.48,0.65,0.82,0.74,0.68,1824,353,2005,172.00,0.82,psb_hack_lama_v541


### Выбор лучшей конфигурации на oot

In [5]:
print('Лучашя модель по метрика')
display(all_metrics.loc[(all_metrics['dataset']=='oot_val') & (all_metrics['group']=='overall')][main_metrics] \
              .applymap(lambda x: all_metrics.iloc[int(x),-1]))
print()
oot = all_metrics.loc[all_metrics['dataset']=='oot_val'] \
              .groupby(['group','measurement'])[main_metrics].idxmin().applymap(lambda x: all_metrics.iloc[int(x),-1])
print('лучшая модель для каждой группы')
display(oot)

Лучашя модель по метрика


,precision,recall,f1,roc-auc original,roc-auc_hack
10,psb_hack_lama_v541,psb_hack_lama_v541,psb_hack_lama_v541,psb_hack_lama_v541,psb_hack_lama_v541



лучшая модель для каждой группы


precision  \
group          measurement                                      
has_prepayment 0                           psb_hack_lama_v541   
               1                           psb_hack_lama_v541   
region         1                           psb_hack_lama_v541   
               2                           psb_hack_lama_v541   
Источник       ostrovok.ru                 psb_hack_lama_v541   
               Бронирование из экстранета  psb_hack_lama_v541   
               Официальный сайт            psb_hack_lama_v541   
               Программа лояльности        psb_hack_lama_v541   
               Яндекс.Путешествия          psb_hack_lama_v541   

                                                       recall  \
group          measurement                                      
has_prepayment 0                           psb_hack_lama_v541   
               1                           psb_hack_lama_v541   
region         1                           psb_hack_lama_v541   
               2                           psb_hack_lama_v541   
Источник       ostrovok.ru                 psb_hack_lama_v541   
               Бронирование из экстранета  psb_hack_lama_v541   
               Официальный сайт            psb_hack_lama_v541   
               Программа лояльности        psb_hack_lama_v541   
               Яндекс.Путешествия          psb_hack_lama_v541   

                                                           f1  \
group          measurement                                      
has_prepayment 0                           psb_hack_lama_v541   
               1                           psb_hack_lama_v541   
region         1                           psb_hack_lama_v541   
               2                           psb_hack_lama_v541   
Источник       ostrovok.ru                 psb_hack_lama_v541   
               Бронирование из экстранета  psb_hack_lama_v541   
               Официальный сайт            psb_hack_lama_v541   
               Программа лояльности        psb_hack_lama_v541   
               Яндекс.Путешествия          psb_hack_lama_v541   

                                             roc-auc original  \
group          measurement                                      
has_prepayment 0                           psb_hack_lama_v541   
               1                           psb_hack_lama_v541   
region         1                           psb_hack_lama_v541   
               2                           psb_hack_lama_v541   
Источник       ostrovok.ru                 psb_hack_lama_v541   
               Бронирование из экстранета  psb_hack_lama_v541   
               Официальный сайт            psb_hack_lama_v541   
               Программа лояльности        psb_hack_lama_v541   
               Яндекс.Путешествия          psb_hack_lama_v541   

                                                 roc-auc_hack  
group          measurement                                     
has_prepayment 0                           psb_hack_lama_v541  
               1                           psb_hack_lama_v541  
region         1                           psb_hack_lama_v541  
               2                           psb_hack_lama_v541  
Источник       ostrovok.ru                 psb_hack_lama_v541  
               Бронирование из экстранета  psb_hack_lama_v541  
               Официальный сайт            psb_hack_lama_v541  
               Программа лояльности        psb_hack_lama_v541  
               Яндекс.Путешествия          psb_hack_lama_v541

### Выбор лучшей конфигурации на val

In [6]:
print('Лучашя модель по метрикам')
display(all_metrics.loc[(all_metrics['dataset']=='oof_val') & (all_metrics['group']=='overall')][main_metrics] \
              .applymap(lambda x: all_metrics.iloc[int(x),-1]))
print()
oof = all_metrics.loc[all_metrics['dataset']=='oof_val'] \
              .groupby(['group','measurement'])[main_metrics].idxmin().applymap(lambda x: all_metrics.iloc[int(x),-1])
print('лучшая модель для каждой группы')
display(oot)

Лучашя модель по метрикам


,precision,recall,f1,roc-auc original,roc-auc_hack
20,psb_hack_lama_v541,psb_hack_lama_v541,psb_hack_lama_v541,psb_hack_lama_v541,psb_hack_lama_v541



лучшая модель для каждой группы


precision  \
group          measurement                                      
has_prepayment 0                           psb_hack_lama_v541   
               1                           psb_hack_lama_v541   
region         1                           psb_hack_lama_v541   
               2                           psb_hack_lama_v541   
Источник       ostrovok.ru                 psb_hack_lama_v541   
               Бронирование из экстранета  psb_hack_lama_v541   
               Официальный сайт            psb_hack_lama_v541   
               Программа лояльности        psb_hack_lama_v541   
               Яндекс.Путешествия          psb_hack_lama_v541   

                                                       recall  \
group          measurement                                      
has_prepayment 0                           psb_hack_lama_v541   
               1                           psb_hack_lama_v541   
region         1                           psb_hack_lama_v541   
               2                           psb_hack_lama_v541   
Источник       ostrovok.ru                 psb_hack_lama_v541   
               Бронирование из экстранета  psb_hack_lama_v541   
               Официальный сайт            psb_hack_lama_v541   
               Программа лояльности        psb_hack_lama_v541   
               Яндекс.Путешествия          psb_hack_lama_v541   

                                                           f1  \
group          measurement                                      
has_prepayment 0                           psb_hack_lama_v541   
               1                           psb_hack_lama_v541   
region         1                           psb_hack_lama_v541   
               2                           psb_hack_lama_v541   
Источник       ostrovok.ru                 psb_hack_lama_v541   
               Бронирование из экстранета  psb_hack_lama_v541   
               Официальный сайт            psb_hack_lama_v541   
               Программа лояльности        psb_hack_lama_v541   
               Яндекс.Путешествия          psb_hack_lama_v541   

                                             roc-auc original  \
group          measurement                                      
has_prepayment 0                           psb_hack_lama_v541   
               1                           psb_hack_lama_v541   
region         1                           psb_hack_lama_v541   
               2                           psb_hack_lama_v541   
Источник       ostrovok.ru                 psb_hack_lama_v541   
               Бронирование из экстранета  psb_hack_lama_v541   
               Официальный сайт            psb_hack_lama_v541   
               Программа лояльности        psb_hack_lama_v541   
               Яндекс.Путешествия          psb_hack_lama_v541   

                                                 roc-auc_hack  
group          measurement                                     
has_prepayment 0                           psb_hack_lama_v541  
               1                           psb_hack_lama_v541  
region         1                           psb_hack_lama_v541  
               2                           psb_hack_lama_v541  
Источник       ostrovok.ru                 psb_hack_lama_v541  
               Бронирование из экстранета  psb_hack_lama_v541  
               Официальный сайт            psb_hack_lama_v541  
               Программа лояльности        psb_hack_lama_v541  
               Яндекс.Путешествия          psb_hack_lama_v541